In [1]:
import os
os.getcwd()

'C:\\Users\\AdminUser'

In [16]:
from io import BytesIO

In [7]:
import matplotlib
import pandas as pd
import numpy as np
import json
import math
import urllib.request
import dateutil.parser
import dateutil.rrule
import datetime
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import collections
from textwrap import wrap

In [4]:
matplotlib.rcParams.update({
    'font.size':13,
    'timezone': 'Europe/London'
})

In [5]:
# Used across all of the plots 
dateToday = datetime.datetime.combine(datetime.date.today(),datetime.datetime.min.time())

# How many seconds between readings in trhe files that are generated. 900s = 15 mins data
resampleFrequency = 3600

In [10]:
# These timestamps are in ISO8601 format. You can Google this for more information.
# YYYY-MM-DDTHH:MM:SSZ - always in UTC, no British Summer Time
# Be aware that you probably need to request a little bit more than you want, because
# otherwise there could be gaps at the start and end where the value didn't change...
requestFrom = '2019-12-31T23:00:00Z' # To include the end of the previous working week...
requestTo = '2020-01-01T00:00:00Z'

In [33]:
usbRequestBase = 'https://api.usb.urbanobservatory.ac.uk/api/v2.0a/sensors/entity'

# Fetch a list of all the entities in the Urban Sciences Building
usbResources = []

requestPage = 1
response = None

filterMetrics = ['Room Temperature']
filterUnits = []

while response is None or len(response) > 1:
  print('Fetching page %u...' % requestPage)

  response = json.loads(
      urllib.request.urlopen(
        '%s?page=%u' % (usbRequestBase, requestPage)
      ).read().decode('utf-8')
  )['items']

  requestPage = requestPage + 1

  for entity in response:
    for feed in entity['feed']:
      for timeseries in feed['timeseries']:
        if 'latest' in timeseries and 'error' in timeseries['latest']:
          # No access to some of the personal data timeseries
          continue
        for link in timeseries['links']:
          if (link['rel'] == 'archives' and \
            'latest' in timeseries):

            if feed['metric'] in filterMetrics or \
               timeseries['unit']['name'] in filterUnits:
              usbResources.append({
                  'entity': entity['name'],
                  'floor': entity['meta']['buildingFloor'],
                  
                  #'bmsId': feed['brokerage'][0]['sourceId'],
                  'metric': feed['metric'],
                  #'units': timeseries['unit']['name'],
                  'timeseriesIri': link['href']
              })

print('Discovered %u entities.' % len(usbResources))

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Fetching page 28...
Fetching page 29...
Fetching page 30...
Fetching page 31...
Fetching page 32...
Fetching page 33...
Fetching page 34...
Fetching page 35...
Fetching page 36...
Fetching page 37...
Fetching page 38...
Fetching page 39...
Fetching page 40...
Fetching page 41...
Fetching page 42...
Fetching page 43...
Fetching page 44...
Fetching page 45...
Fetching page 46...
Fetching page 47...
Fetching page 48...
Fetching page 49...
Fetching page 50...
Fetching 

In [14]:
type(usbResources)
usbResources

[{'entity': 'Urban Sciences Building: Floor 1 Ambient Kitchen',
  'bmsId': '117_AmbientKitchen_1.018_RoomT',
  'metric': 'Room Temperature',
  'units': 'degrees celsius',
  'timeseriesIri': 'https://api.usb.urbanobservatory.ac.uk/api/v2/sensors/timeseries/76a0b94e-b721-41a7-b8ff-f53a7f2dc517/historic'},
 {'entity': 'Urban Sciences Building: Floor 1 Collaboration',
  'bmsId': '105_Collaboration_1.005_RoomT',
  'metric': 'Room Temperature',
  'units': 'degrees celsius',
  'timeseriesIri': 'https://api.usb.urbanobservatory.ac.uk/api/v2/sensors/timeseries/3a0a0c64-362d-4669-8091-9b21102d6f8c/historic'},
 {'entity': 'Urban Sciences Building: Floor 1 Collaboration',
  'bmsId': '109_Collaboration_1.010_RoomT',
  'metric': 'Room Temperature',
  'units': 'degrees celsius',
  'timeseriesIri': 'https://api.usb.urbanobservatory.ac.uk/api/v2/sensors/timeseries/3496654a-21e8-41cb-8759-e391e29402c5/historic'},
 {'entity': 'Urban Sciences Building: Floor 1 DIG Zone 1',
  'bmsId': '118_DIG_1.024_Z1_Roo

In [32]:
usbMetadata[usbMetadata['floor'] == '6']['timeseriesIri']

327    https://api.usb.urbanobservatory.ac.uk/api/v2/...
328    https://api.usb.urbanobservatory.ac.uk/api/v2/...
329    https://api.usb.urbanobservatory.ac.uk/api/v2/...
330    https://api.usb.urbanobservatory.ac.uk/api/v2/...
331    https://api.usb.urbanobservatory.ac.uk/api/v2/...
332    https://api.usb.urbanobservatory.ac.uk/api/v2/...
333    https://api.usb.urbanobservatory.ac.uk/api/v2/...
334    https://api.usb.urbanobservatory.ac.uk/api/v2/...
335    https://api.usb.urbanobservatory.ac.uk/api/v2/...
336    https://api.usb.urbanobservatory.ac.uk/api/v2/...
337    https://api.usb.urbanobservatory.ac.uk/api/v2/...
338    https://api.usb.urbanobservatory.ac.uk/api/v2/...
339    https://api.usb.urbanobservatory.ac.uk/api/v2/...
340    https://api.usb.urbanobservatory.ac.uk/api/v2/...
341    https://api.usb.urbanobservatory.ac.uk/api/v2/...
342    https://api.usb.urbanobservatory.ac.uk/api/v2/...
343    https://api.usb.urbanobservatory.ac.uk/api/v2/...
344    https://api.usb.urbanobs

In [24]:
usbMetadata[usbMetadata['floor'] == 'G']

,entity,floor,bmsId,timeseriesIri
385,Urban Sciences Building: Floor G Back Projection,G,009_BackProjection_G.011_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
386,Urban Sciences Building: Floor G Cafe Store,G,055_CafeStore_G.069_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
387,Urban Sciences Building: Floor G Computer Work...,G,032_ComputerWorkshop_G.044_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
388,Urban Sciences Building: Floor G Cyber Physical,G,004_CyberPhys_G.006_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
389,Urban Sciences Building: Floor G Decision Theatre,G,008_DecisionTheatre_G.010_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
390,Urban Sciences Building: Floor G Draught Lobby,G,065_DraughtLobby_G.081_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
391,Urban Sciences Building: Floor G EPTCell,G,019_EPTCell_G.020_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
392,Urban Sciences Building: Floor G EPTControl,G,020_EPTControl_G.021_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
393,Urban Sciences Building: Floor G Event Store,G,069_EventStore_G.004_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
394,Urban Sciences Building: Floor G Female Showers,G,040_FemaleShowers_G.054_RoomT,https://api.usb.urbanobservatory.ac.uk/api/v2/...
